In [8]:
from ufunctions import *

In [9]:
dfQuote = GetQuote(ticker='SAVE', start='2018-01-01')

[*********************100%***********************]  1 of 1 downloaded


In [10]:
# dfQuote

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,44.70,45.61,44.09,45.60,45.60,899200
2018-01-03,45.54,45.54,43.47,44.14,44.14,1413800
2018-01-04,43.94,44.25,43.33,43.50,43.50,1823600
2018-01-05,43.75,43.89,43.45,43.63,43.63,680700
2018-01-08,43.75,43.75,42.89,43.23,43.23,1059900
2018-01-09,45.19,45.19,43.36,43.59,43.59,1745600
2018-01-10,44.07,44.93,43.66,44.91,44.91,1461400
2018-01-11,45.83,48.02,45.54,47.57,47.57,2478700
2018-01-12,47.70,48.81,47.25,48.26,48.26,1346700
